# 1 Training Setup

In [2]:
# import necessary library for training
import numpy as np 
import pandas as pd 
from subprocess import check_output
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#import xgboost as xgb
#from sklearn import cross_validation
import scipy.stats as stats
import gc

In [3]:
train = pd.read_csv('s3://trendzmarket/train.csv') 

In [5]:
train.head()

,Unnamed: 0,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,1,9,1122,1,1074,1290922,2233,2,39.88,0,0.0,2
1,2,6,1544,4,6601,2163331,1120,2,42.14,0,0.0,2
2,3,6,1476,4,4902,2305065,43194,9,122.76,0,0.0,9
3,4,8,1346,1,1105,529868,43209,4,27.04,0,0.0,4
4,5,7,1623,1,2133,178265,37361,6,37.50,0,0.0,6


In [6]:
test = pd.read_csv('s3://trendzmarket/test.csv')

In [7]:
# define target variable and drop the previous defined column
train['target'] = train['Demanda_uni_equil']
train.drop(['Demanda_uni_equil'],axis=1, inplace = True)

In [8]:
# label test and train data
train['tst'] = 0
test['tst'] = 1

In [9]:
# concatenate train and test data vertically
data = pd.concat([train,test], axis=0, copy=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [10]:
data.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Unnamed: 0,Venta_hoy,Venta_uni_hoy,id,target,tst
0,1122,1,1290922,0.0,0.0,2233,1074,9,1.0,39.88,2.0,NaN,2.0,0
1,1544,4,2163331,0.0,0.0,1120,6601,6,2.0,42.14,2.0,NaN,2.0,0
2,1476,4,2305065,0.0,0.0,43194,4902,6,3.0,122.76,9.0,NaN,9.0,0
3,1346,1,529868,0.0,0.0,43209,1105,8,4.0,27.04,4.0,NaN,4.0,0
4,1623,1,178265,0.0,0.0,37361,2133,7,5.0,37.50,6.0,NaN,6.0,0


In [11]:
# clear the stored data in the RAM to release space
del train
del test
gc.collect()

140

In [12]:
# label lag column, which is important for feature engineering because sales in previous weeks would affect
# sales this week
for i in range(1,6):
    lag = 'Lag' + str(i)
    print('Lag:',lag)
    
    data1 = data[['Semana','Cliente_ID','Producto_ID','target']]
    data1.loc[:,'Semana'] = data1.loc[:,'Semana'] +i
    data1 = data1.groupby(['Semana','Cliente_ID','Producto_ID']).mean() 
    data1 = data1.reset_index()
    data1.rename(columns={'target': lag}, inplace=True)
    data = pd.merge(data,data1,
                    how='left',
                    left_on=['Semana','Cliente_ID','Producto_ID'], 
                    right_on=['Semana','Cliente_ID','Producto_ID'],
                    left_index=False, right_index=False, sort=True,
                    suffixes=('_x', '_y'), copy=False, )
    del data1
    gc.collect()

Lag: Lag1


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Lag: Lag2
Lag: Lag3
Lag: Lag4
Lag: Lag5


In [13]:
data.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Unnamed: 0,Venta_hoy,Venta_uni_hoy,id,target,tst,Lag1,Lag2,Lag3,Lag4,Lag5
0,2013,1,3473,0.0,0.0,1284,1103,3,646.0,54.36,18.0,NaN,18.0,0,NaN,NaN,NaN,NaN,NaN
1,2012,1,3644,0.0,0.0,43285,2009,3,709.0,52.80,10.0,NaN,10.0,0,NaN,NaN,NaN,NaN,NaN
2,2013,1,5223,0.0,0.0,1278,1210,3,306.0,22.50,5.0,NaN,5.0,0,NaN,NaN,NaN,NaN,NaN
3,1227,1,14684,0.0,0.0,36610,4445,3,438.0,30.80,40.0,NaN,40.0,0,NaN,NaN,NaN,NaN,NaN
4,1227,1,15035,0.0,0.0,1064,1243,3,428.0,16.67,1.0,NaN,1.0,0,NaN,NaN,NaN,NaN,NaN


In [14]:
# create a lag column which is the sum of past 5 weeks sales data
data.loc[:,'TotalLags'] = data.loc[:,'Lag1'] + data.loc[:,'Lag2']+ data.loc[:,'Lag3']+ data.loc[:,'Lag4']+ data.loc[:,'Lag5']

In [15]:
# we do feature engineering for week 8, 9, 10, 11's data, because of recency
data=data[data['Semana']>8]

# 2 Feature Engineering

In [17]:
# feature engineering -1
# calculate the mean of count of Agencia_ID
nAgencia = pd.DataFrame(data.groupby(['Agencia_ID','Semana'])['target'].count())
nAgencia = nAgencia.reset_index()
nAgencia.rename(columns={'target': 'nAgencia'}, inplace=True)
nAgencia = pd.DataFrame(nAgencia.groupby(['Agencia_ID'])['nAgencia'].mean())
nAgencia = nAgencia.reset_index()

In [19]:
data = pd.merge(data, nAgencia, 
                            how='left',
                            left_on=['Agencia_ID'], 
                            right_on=['Agencia_ID'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

In [20]:
# release RAM and clear cache
del nAgencia
gc.collect()

144

In [21]:
print(data.shape[0])

6999384


In [22]:
# feature engineering -2
# calculate the mean of count of Ruta_SAK
nRuta_SAK = pd.DataFrame(data.groupby(['Ruta_SAK','Semana'])['target'].count())
nRuta_SAK = nRuta_SAK.reset_index()
nRuta_SAK.rename(columns={'target': 'nRuta_SAK'}, inplace=True)
nRuta_SAK = pd.DataFrame(nRuta_SAK.groupby(['Ruta_SAK'])['nRuta_SAK'].mean())
nRuta_SAK = nRuta_SAK.reset_index()

In [23]:
data = pd.merge(data, nRuta_SAK, 
                            how='left',
                            left_on=['Ruta_SAK'], 
                            right_on=['Ruta_SAK'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

In [24]:
del nRuta_SAK
gc.collect()
print('merge completo nRuta_SAK')
print(data.shape[0])

merge completo nRuta_SAK
6999384


In [29]:
# feature engineering -3
# calculate the mean of count of Cliente_ID
nCliente_ID = pd.DataFrame(data.groupby(['Cliente_ID','Semana'])['target'].count())
nCliente_ID = nCliente_ID.reset_index()
nCliente_ID.rename(columns={'target': 'nCliente_ID'}, inplace=True)
nCliente_ID = pd.DataFrame(nCliente_ID.groupby(['Cliente_ID'])['nCliente_ID'].mean())
nCliente_ID = nCliente_ID.reset_index()

In [30]:
data = pd.merge(data, nCliente_ID, 
                            how='left',
                            left_on=['Cliente_ID'], 
                            right_on=['Cliente_ID'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

In [31]:
del nCliente_ID
gc.collect()
print('merge completo nCliente_ID')
print(data.shape[0])

merge completo nCliente_ID
6999384


In [33]:
data.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Unnamed: 0,Venta_hoy,...,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID_x,nCliente_ID_y
0,2655,2,26,NaN,NaN,31518,4189,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,2655,2,26,NaN,NaN,31520,4189,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,2061,2,26,NaN,NaN,34210,7212,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,2061,2,26,NaN,NaN,34785,7212,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,2061,2,26,NaN,NaN,34786,7212,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [34]:
# feature engineering -4
# calcuate the mean of count of Producto_ID
nProducto_ID = pd.DataFrame(data.groupby(['Producto_ID','Semana'])['target'].count())
nProducto_ID = nProducto_ID.reset_index()
nProducto_ID.rename(columns={'target': 'nProducto_ID'}, inplace=True)
nProducto_ID = pd.DataFrame(nProducto_ID.groupby(['Producto_ID'])['nProducto_ID'].mean())
nProducto_ID = nProducto_ID.reset_index()

In [35]:
data = pd.merge(data, nProducto_ID, 
                            how='left',
                            left_on=['Producto_ID'], 
                            right_on=['Producto_ID'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 


In [36]:
del nProducto_ID
gc.collect()
print('merge completo nProducto_ID')
print(data.shape[0])

merge completo nProducto_ID
6999384


# Data Transformation

In [ ]:
# sagemaker has specific requirement for training data format
# Amazon SageMaker XGBoost can train on data in either a CSV or LibSVM format. 
# For this example, we use CSV. It should have the following:
## Have the predictor variable in the first column
## Not have a header row

In [37]:
data.replace(np.nan,0, inplace=True)

In [38]:
train = data[data['tst']==0]
predict = data[data['tst']==1]

In [40]:
train.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Unnamed: 0,Venta_hoy,...,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID_x,nCliente_ID_y,nProducto_ID
24594,1336,1,554912,0.00,0.0,73,1071,9,572.0,63.96,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333,0.333333
47074,1212,1,371367,0.00,0.0,303,1617,9,865.0,9.08,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333,0.333333
85951,1153,1,1774446,0.00,0.0,328,1603,9,130.0,24.45,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333,0.333333
154510,22362,1,4479899,18.98,1.0,972,1245,9,633.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.333333,0.666667
157691,1217,1,4710196,0.00,0.0,972,1140,9,559.0,18.98,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333,0.666667


In [42]:
# change to log for kaggle submission requirement
train['target'] = np.log(train['target'] + 1)
y = train['target']
X = train[["Agencia_ID","Canal_ID","Cliente_ID","Producto_ID","Ruta_SAK","Lag1",\
           "Lag2","Lag3","Lag4","Lag5","TotalLags","nAgencia","nRuta_SAK","nCliente_ID_x","nProducto_ID"]]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
y.head()

24594     0.625800
47074     0.554618
85951     0.625800
154510    0.000000
157691    0.423036
Name: target, dtype: float64

In [137]:
X.head()

,Agencia_ID,Canal_ID,Cliente_ID,Producto_ID,Ruta_SAK,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID_x,nProducto_ID
24594,1336,1,554912,73,1071,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
47074,1212,1,371367,303,1617,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
85951,1153,1,1774446,328,1603,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
154510,22362,1,4479899,972,1245,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.666667
157691,1217,1,4710196,972,1140,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.666667


In [143]:
dataset = pd.concat([y, X], axis = 1)

In [144]:
dataset.head()

,target,Agencia_ID,Canal_ID,Cliente_ID,Producto_ID,Ruta_SAK,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID_x,nProducto_ID
24594,0.625800,1336,1,554912,73,1071,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
47074,0.554618,1212,1,371367,303,1617,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
85951,0.625800,1153,1,1774446,328,1603,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
154510,0.000000,22362,1,4479899,972,1245,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.666667
157691,0.423036,1217,1,4710196,972,1140,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.666667


In [145]:
# split training data into train, validation and test set
train_data, validation_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset)), int(0.9 * len(dataset))])

In [146]:
train_data.fillna(0, inplace = True)

In [147]:
validation_data.fillna(0,inplace = True)

In [148]:
validation_data

,target,Agencia_ID,Canal_ID,Cliente_ID,Producto_ID,Ruta_SAK,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID_x,nProducto_ID
157691,0.423036,1217,1,4710196,972,1140,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.666667
6284013,0.891821,1445,1,189885,43207,1141,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
4428020,0.554618,1626,1,2483771,35309,2106,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,1.000000
3309337,0.423036,1123,1,1503755,30552,2103,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.333333
3418910,0.423036,2030,1,1431419,31423,1171,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.333333
6895103,0.625800,1245,1,333015,47840,1011,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.666667
5458564,0.706395,1427,1,206613,43038,5503,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
154510,0.000000,22362,1,4479899,972,1245,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.666667
6336517,0.423036,1419,1,767885,43210,1192,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
1702163,0.706395,1622,1,1737893,1242,1160,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,1.000000


In [155]:
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [62]:
del train
gc.collect()

891

# Model Training

In [67]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3
import os

In [169]:
s3_input_train  = sagemaker.s3_input(s3_data='s3://trendzmarket/segmaker/train/train.csv', content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://trendzmarket/segmaker/validation/validation.csv', content_type='csv')

In [94]:
from sagemaker.amazon.amazon_estimator import get_image_uri
sess = sagemaker.Session()
region = boto3.Session().region_name
container = get_image_uri(region, 'xgboost')

	get_image_uri(region, 'xgboost', '0.90-1').


In [95]:
container

'811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:1'

In [154]:
train_data.dtypes

target           float64
Agencia_ID         int64
Canal_ID           int64
Cliente_ID         int64
Producto_ID        int64
Ruta_SAK           int64
Lag1             float64
Lag2             float64
Lag3             float64
Lag4             float64
Lag5             float64
TotalLags        float64
nAgencia         float64
nRuta_SAK        float64
nCliente_ID_x    float64
nProducto_ID     float64
dtype: object

In [97]:
role = get_execution_role()

In [99]:
role

'arn:aws:iam::097914252455:role/service-role/AmazonSageMaker-ExecutionRole-20191208T142674'

In [170]:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path= 's3://trendzmarket/segmaker/train/',
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=10,\
                        objective='reg:linear', \
                        eta =0.2,\
                        gamma=0,\
                        silence = 
                        min_child_weight=1,\
                        max_delta_step=0,\
                        subsample=0.85, \
                        colsample_bytree=0.7,\
                        colsample_bylevel=1,\
                        alpha=0,\
                        scale_pos_weight=1,\
                        num_round = 100,\
                        seed=1440)

xgb.fit({'train': s3_input_train,'validation':s3_input_validation})

2019-12-09 00:43:33 Starting - Starting the training job...
2019-12-09 00:43:34 Starting - Launching requested ML instances......
2019-12-09 00:44:38 Starting - Preparing the instances for training...
2019-12-09 00:45:25 Downloading - Downloading input data...
2019-12-09 00:45:46 Training - Downloading the training image.Arguments: train
[2019-12-09:00:46:06:INFO] Running standalone xgboost training.
[2019-12-09:00:46:06:INFO] File size need to be processed in the node: 0.02mb. Available memory size in the node: 8534.04mb
[2019-12-09:00:46:06:INFO] Determined delimiter of CSV input is ','
[00:46:06] S3DistributionType set as FullyReplicated
[00:46:06] 93x15 matrix with 1395 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2019-12-09:00:46:06:INFO] Determined delimiter of CSV input is ','
[00:46:06] S3DistributionType set as FullyReplicated
[00:46:06] 26x15 matrix with 390 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&deli

In [177]:
test_data.head(20)

,target,Agencia_ID,Canal_ID,Cliente_ID,Producto_ID,Ruta_SAK,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID_x,nProducto_ID
3722681,0.625800,4062,2,4680076,32303,2598,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
5598926,0.625800,1222,1,52999,43064,4455,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,1.000000
2041598,0.771165,1121,1,390789,1278,1419,0.0,0.0,0.0,0.0,0.0,0.0,1.333333,0.333333,0.333333,1.000000
4687191,0.423036,22362,1,4244077,35727,1057,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.333333
3371678,0.000000,2030,1,375335,31298,5001,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.333333
1887364,0.554618,1227,1,2366966,1250,1234,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,1.333333
3102519,0.863863,1911,1,107665,8921,1158,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.333333
4628970,0.625800,2045,1,1485742,35651,1157,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.333333,0.666667
749147,0.423036,2235,1,2013128,1150,1202,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.333333,0.333333
3382553,0.554618,1967,1,631787,31311,2101,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.333333,0.333333


# Model Predicting

In [173]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------------------------------------------------------------------------------------------------!

In [180]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [181]:
# take first 20 rows to get an quick insight
def predict(data, rows=20):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [182]:
predictions = predict(test_data.as_matrix()[:, 1:])
predictions

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


array([0.69602442, 0.64289719, 0.58409405, 0.59537643, 0.64102364,
       0.84037292, 0.57066989, 0.56673837, 0.64797705, 0.71766329,
       0.77927375, 0.66825068, 0.75400859, 0.61999738])

In [192]:
test_data["target"][0:19]

3722681    0.625800
5598926    0.625800
2041598    0.771165
4687191    0.423036
3371678    0.000000
1887364    0.554618
3102519    0.863863
4628970    0.625800
749147     0.423036
3382553    0.554618
6563873    0.798928
1219302    0.625800
2042081    0.706395
5508585    0.706395
Name: target, dtype: float64

In [193]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(test_data["target"][0:19], predictions))

In [194]:
rms

0.23006829709296428